In [1]:
#imports
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset

import numpy as np
from glob import glob
from PIL import Image
import pandas as pd
import os

ngpu=1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Data Preparation
- Download of Dataset
- Post Mega/Download Link for Train abd Validation data of Imagenet 2012 (Obtained from Kaggle)
    - Validation Data: [Mega Link](https://mega.nz/#!yDoTDIyD!RjN6OBA92-KLpNqDeLS3OzwmAYesEbTsiQat9hT6p6s)
    - Trainning Data: [Mega Link](https://mega.nz/#!vKY0WSDa!4aibnBkiXUrO9MkhQlLGXac7wLF5HY7O4LzfdFEaeQU) **P.S**: Randomly Sampled 10 instances from each target class as described in the paper.
- If link fails to work use the following Colab notebook to generate your own subset of trainning examples. [Link](https://colab.research.google.com/drive/1LbZBfgqntWb3HuC3UFyF_FvwnHtd1xTA)
- Setting up of Folder Structure
For Easier handling and reproducibility of results download from mega link 


In [2]:
dataset_path=r'ILSVRC/'
train_dataset_path=dataset_path+'train'
test_dataset_path=dataset_path+'valid'
print(dataset_path,train_dataset_path,test_dataset_path)

ILSVRC/ ILSVRC/train ILSVRC/valid


In [3]:
# Preparation of Labels 
label_dict={}
label_idx={}
with open('ILSVRC/LOC_synset_mapping.txt') as file:
    lines=file.readlines()
    for idx,line in enumerate(lines):
        label,actual =line.strip('\n').split(' ',maxsplit=1)
        label_dict[label]=actual
        label_idx[label]=idx
        

In [4]:
# Experiment Description :

In [5]:
# Dataset and Dataloaders
class CustomDataset(Dataset):
    def __init__(self, subset, root_dir, transform=None):
        self.root_dir=root_dir
        self.transform=transform
       
        self.subset=subset
        if self.subset=='train':
            data_dir=os.path.join(self.root_dir,self.subset)
            self.images_fn=glob(f'{data_dir}/*/*')
            self.labels=[fn.split('\\')[1] for fn in self.images_fn]
        elif subset =='valid':
            df=pd.read_csv('ILSVRC/LOC_val_solution.csv')
            df['label']=df['PredictionString'].str.split(' ',n=1,expand=True)[0]
            df=df.drop(columns=['PredictionString'])
            self.images_fn='ILSVRC/valid/'+df['ImageId'].values+'.JPEG'
            self.labels=df['label']
        else:
            raise ValueError
        print(f" Number of instances in {self.subset} subset of Dataset: {len(self.images_fn)}")       

    def __getitem__(self,idx):
        fn=self.images_fn[idx]
        label=self.labels[idx]
        image=Image.open(fn)
        if image.getbands()[0] == 'L':
            image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)    
#         print(type(image))
        return image,label_idx[label]

        
    
    def __len__(self):
        return len(self.images_fn)
        

In [6]:
# transforms
size=128
mean = [103.939, 116.779, 123.68]


preprocess=transforms.Compose([transforms.Resize((size,size)),
                               transforms.ToTensor(),
                               transforms.Normalize(mean,(0.5, 0.5, 0.5))])

In [7]:
data_train=CustomDataset(subset='train',root_dir=dataset_path,transform=preprocess) # Need to Fix train Data
data_test=CustomDataset(subset='valid',root_dir=dataset_path,transform=preprocess)

 Number of instances in train subset of Dataset: 10000
 Number of instances in valid subset of Dataset: 50000


from tqdm import tqdm_notebook as tqdm
for images,labels in tqdm(DataLoader(data_test,batch_size=100)):
    print(images.shape,len(labels))
    break

import pandas as pd

df=pd.read_csv('ILSVRC/LOC_val_solution.csv')
df['label']=df['PredictionString'].str.split(' ',n=1,expand=True)[0]
df=df.drop(columns=['PredictionString'])


# Proposed Approach

![Proposed approach](resources\\nag.png)

- **Core idea is to model the distribution of universal adversarial perturbations for a given classifier.**
- The image shows a batch of B random vectors {z}<sub>B</sub> transforming into perturbations {delta}<sub>B</sub> by G which get added to the batch of data samples {x}<sub>B</sub>.
- The top portion shows adversarial batch (X<sub>A</sub>), bottom portion shows shuffled adversarial batch (X<sub>S</sub>) and middle portion shows the benign batch (X<sub>B</sub>). The Fooling objective Lf (eq. 2) and Diversity objective Ld (eq. 3) constitute the loss. 
### Note
- Note that the target CNN (f) is a trained classifier and its parameters are not updated during the proposed training. On the other hand, the parameters of generator (G) are randomly initialized and learned through backpropagating the loss. (Best viewed in color).

# Choice of Hyperparameters
- The architecture of the generator consists of 5 deconv layers. The final deconv layer is followed by a tanh non-linearity and scaling by epsillon (10)


In [8]:
# epsillon=10
# batch_size=32
# latent_dim = 10
# img_h,img_w,img_c=(224,224,3)

latent_dim=10
arch='googlenet'
archs=['vgg-f','vgg16','vgg19','googlenet','resnet50','resnet152']
if arch in ['vgg16','vgg19','vgg-f','googlenet']:
    bs=64
elif arch in ['resnet50','resnet152']:
    bs=32
else:
    raise ValueError(f'Architecture type not supported. Please choose one from the following {archs}')
bs
bs=4 # OOM Error


# Generator 
- Architecture of our generator (G) unchanged for different target CNN architectures

In [9]:
from torch import nn

![DCGAN](resources/DCGAN.png)

In [10]:
# Effect of ConvTranspose2d : combination of upsampling and convolution layers is equal to a strided
# convolutional layer. increase the spatial resolution of the tensor

In [11]:
ngf=128
nz= latent_dim
nc=3 # Number of Channels

class AdveraryGenerator(nn.Module):
    def __init__(self):
        super(AdveraryGenerator, self).__init__()
        self.main = nn.Sequential(
        nn.ConvTranspose2d( nz, 1024, 4, 1, 0, bias=False),
        nn.BatchNorm2d(1024),
        nn.ReLU(True),
        # state size. (ngf*8) x 4 x 4
        nn.ConvTranspose2d(1024, 512, 4, 2, 1, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(True),
        # state size. (ngf*4) x 8 x 8
        nn.ConvTranspose2d( 512, 256, 4, 2, 1, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        # state size. (ngf*2) x 16 x 16
        nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),
        # state size. (ngf) x 32 x 32
        nn.ConvTranspose2d( 128, nc, 4, 4, 0, bias=False),
        nn.Tanh()
        # state size. (nc) x 64 x 64
        )
    #     def __call__(self):
    # Dont Override the __call__ method. Pytorch does forward and backward hooks required, 
    # Always use forward method to avoid any issues


    def forward(self, x):
        return self.main(x)


In [12]:
adversarygen=AdveraryGenerator().to(device)


In [14]:
soft=nn.Softmax()
soft(clean_preds)

C:\Users\Gokkulnath\Anaconda3\envs\fastai\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([[2.2222e-05, 2.9776e-03, 1.0532e-04,  ..., 1.3071e-04, 1.0350e-03,
         7.7704e-03],
        [1.2753e-05, 1.7874e-05, 4.1048e-04,  ..., 2.0979e-04, 6.4641e-04,
         1.8757e-03],
        [8.6980e-05, 2.1691e-04, 1.3905e-05,  ..., 3.9865e-05, 6.9964e-04,
         2.0398e-04],
        [3.9080e-03, 1.6218e-04, 9.7747e-04,  ..., 6.9027e-04, 1.1081e-03,
         9.8989e-05]], device='cuda:0')

In [18]:
# Train Loop 

# Model Architecture
import torchvision.models as tvm
f=tvm.resnet50(pretrained=True)
f.to(device)#.eval()


train_dl=DataLoader(data_train,batch_size=bs,shuffle=False)

loss= nn.CrossEntropyLoss()


from tqdm import tqdm_notebook as tqdm
for i,data in enumerate(train_dl,0):
    images=data[0].to(device)
    labels=data[1].to(device)

    noise=adversarygen(torch.randn(bs, nz, 1, 1, device=device))
#     print(images.shape,noise.shape,labels)
    
    with torch.no_grad() :
        # XB = images
        clean_preds=f(images)

        #XA = images+noise
        preds_XA= f(images+noise)
        '''
        From the Paper -->
        We also randomly shuffle the perturbations ensuring no perturbation remains in its original index in the batch
        '''
        #XS = images+ noise[torch.randperm(bs)] # Shuffling Logic ? Across the batch dimension or within the image ? 1. Within Image 
        preds_XS=f(images+ noise[torch.randperm(bs)])
    
#     print(XA.shape,XB.shape,XS.shape)
    
    print(preds_XA.shape,clean_preds.shape,preds_XS.shape)
    print(i,torch.argmax(soft(preds_XA),dim=1),torch.argmax(soft(clean_preds),dim=1),torch.argmax(soft(preds_XS),dim=1))

    break

torch.Size([4, 1000]) torch.Size([4, 1000]) torch.Size([4, 1000])


C:\Users\Gokkulnath\Anaconda3\envs\fastai\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0 tensor([ 60, 126, 680, 421], device='cuda:0') tensor([406, 570, 428, 506], device='cuda:0') tensor([558, 676, 738, 577], device='cuda:0')


In [ ]:
# Fooling Objective
- 
[406, 570, 428, 506]

In [ ]:
fooling_obj= -1 * torch.log(1-clean_preds)

# print()

In [ ]:
%debug

# To do : Need to Write a Custom Model from scratch for VGG -F  and load weights from caffe model
- Refer Here : https://github.com/val-iisc/nag/blob/83564eb4a8b5177660e2f6566dd63faa16f76773/nets/vgg_f.py
- https://github.com/val-iisc/nag/blob/83564eb4a8b5177660e2f6566dd63faa16f76773/misc/convert_weights.py
- Here VGGF refers to VGG-Face model  http://www.vlfeat.org/matconvnet/pretrained/. 
- How we can use that to classify Imagenet ?
- load caffe prototxt and weights directly in pytorch --> https://github.com/marvis/pytorch-caffe
- Convert Caffe models to Pytorch : https://github.com/vadimkantorov/caffemodel2pytorch
#### Check this link for Conversion Tutorial : [Link](https://colab.research.google.com/drive/1i2dq6qctPvrLREhKOZNNBsNfKuaS0HYQ)

In [ ]:
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=0.001)


https://pytorch.org/docs/stable/nn.html?highlight=convtranspose2d#torch.nn.ConvTranspose2d

# Futute Scope of Work 
# Variational Auto-Encoders (VAE) --> Not Done ? Why ?